In [312]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from io import StringIO

In [313]:
# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_df = pd.read_csv(school_data_to_load)
student_df = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_student_df = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])

In [314]:
# Merging two DataFrames
school_stus_df=school_student_df.drop(["student_name","gender","grade","School ID"],axis=1)
school_stus_df.head()

,Student ID,school_name,reading_score,math_score,type,size,budget
0,0,Huang High School,66,79,District,2917,1910635
1,1,Huang High School,94,61,District,2917,1910635
2,2,Huang High School,90,60,District,2917,1910635
3,3,Huang High School,67,58,District,2917,1910635
4,4,Huang High School,97,84,District,2917,1910635


In [315]:
#1. --------District Summary DataFrame

# Calculating parameters for the summary

#Calculate the total number of schools
total_schools=school_df['school_name'].count()

#Calculate the total number of students
tot_students= student_df['student_name'].count()

#Calculate the total budget per school
total_budget= school_df["budget"].sum()

#Calculate the average math score 
average_math_score=student_df["math_score"].mean()

#Calculate the average reading score
average_reading_score=student_df["reading_score"].mean()

#Calculate the number of students with a passing math score
students_pass_math_score= school_student_df.loc[school_student_df["math_score"]>=70] 
#Calculate the percentage of students with a passing math score
perc_stu_pass_math= len(students_pass_math_score)/tot_students *100

#Calculate the number of students with a passing reading score
students_pass_reading_score=school_student_df.loc[school_student_df["reading_score"]>=70]
#Calculate the percentage of students with a passing reading score
perc_stu_pass_reading= len(students_pass_reading_score)/tot_students *100

# Calculate the percentage of students who passed math and reading (% Overall Passing)
overall_pass_math_reading= school_student_df.loc[(school_student_df["math_score"]>=70) &(school_student_df["reading_score"]>=70)].count()["student_name"]
overall_pass_grade=overall_pass_math_reading/tot_students*100


In [316]:
# District Summary DataFrame

# Create a dataframe to hold the above results
dist_summary=pd.DataFrame({
    "Total Schools":[total_schools],"Total Students":[tot_students],
    "Total Budget":[total_budget],"Average Math Score": [average_math_score],"Average Reading Score": [average_reading_score],
    "Passing Math %": [perc_stu_pass_math],"Passing Reading %": [perc_stu_pass_reading], "Overall Passing %":[overall_pass_grade]})
dist_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Passing Math %,Passing Reading %,Overall Passing %
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


In [317]:
# Improve formatting before outputting
dist_summary["Total Budget"] = dist_summary["Total Budget"].map("${0:,.2f}".format)
dist_summary["Total Students"] = dist_summary["Total Students"].map("{0:,.0f}".format)
dist_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Passing Math %,Passing Reading %,Overall Passing %
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [320]:
#2. ----- School Summary

school_index= school_df.set_index('school_name')

# Groupby function on schools
school_stus_index= school_stus_df.set_index('school_name')
school_group= school_stus_index.groupby(['school_name'])
school_group.head()



,Student ID,reading_score,math_score,type,size,budget
school_name,,,,,,
Huang High School,0,66,79,District,2917,1910635
Huang High School,1,94,61,District,2917,1910635
Huang High School,2,90,60,District,2917,1910635
Huang High School,3,67,58,District,2917,1910635
Huang High School,4,97,84,District,2917,1910635
...,...,...,...,...,...,...
Thomas High School,37535,76,76,Charter,1635,1043130
Thomas High School,37536,84,82,Charter,1635,1043130
Thomas High School,37537,80,90,Charter,1635,1043130


In [321]:
# Calculating parameters for the summary


# School Type (District per school)
school_type=school_index["type"] 

# Total_Students per school
total_students= school_index["size"]

# Total Budget per School
school_budget=school_index["budget"] 

# Per student Budget
student_budget= school_budget/total_students

# Averages using Groupby on the reading score per school
average_reading_score = school_group["reading_score"].mean()

# Averages using Groupby on the math score per school
average_math_score = school_group["math_score"].mean()

# % Passing Math
passing_math= students_pass_math_score.groupby("school_name") 
perc_passing_math= passing_math["Student ID"].count()/total_students*100

# % Passing Reading
passing_reading= students_pass_reading_score.groupby("school_name") 
perc_passing_reading= passing_reading["Student ID"].count()/total_students*100

#% Overall Passing (The percentage of students that passed math and reading.)
# Calculate the percentage of students who passed math and reading (% Overall Passing)
overall_pass_m_r= school_student_df.loc[(school_student_df["math_score"]>=70) &(school_student_df["reading_score"]>=70)]
overall_passing=overall_pass_m_r.groupby("school_name").count()["student_name"] /total_students*100


In [323]:
# School Summary DataFrame

schools_summary=pd.DataFrame({
    "School Type":school_type,
    "Total Students":total_students,
    "Total School Budget":school_budget,
    "Per Student Budget":student_budget,
    "Average Math Score":average_math_score,
    "Average Reading Score":average_reading_score,
    "% Passing Math":perc_passing_math,
    "% Passing Reading":perc_passing_reading,
    "Overall Passing Rate":overall_passing})


In [205]:
# Formatting the numbers
schools_summary["Total School Budget"]= schools_summary["Total School Budget"].map("${:,.2f}".format)
schools_summary["Per Student Budget"]= schools_summary["Per Student Budget"].map("${:,.2f}".format)

In [206]:
schools_summary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455


In [207]:
# 4.-------Top Performing Schools (Sorted and displayed the top five performing schools by % overall passing.)

top_schools= schools_summary.loc[schools_summary["Overall Passing Rate"]>90]
top_schools.sort_values(["Overall Passing Rate"], ascending=False).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [208]:
#5.----------- Bottom Performing Schools (By Passing Rate)
bottom_perf_schools= schools_summary.loc[schools_summary["Overall Passing Rate"]<75]
bottom_perf_schools.sort_values(["Overall Passing Rate"], ascending=True).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [324]:
#6.------- Math Scores by Grade
# First using loc to filter per grade, then groupby per school
# Second getting column of math score 

# 9th grade
nineth= student_df.loc[student_df["grade"]=="9th"].groupby(["school_name"])
nineth_math_score=nineth["math_score"].mean()
# 10th grade
tenth= student_df.loc[student_df["grade"]=="10th"].groupby(["school_name"])
tenth_math_score=tenth["math_score"].mean()
# 11th grade
eleventh= student_df.loc[student_df["grade"]=="11th"].groupby(["school_name"])
eleventh_math_score=eleventh["math_score"].mean()
# 12th grade
twelveth= student_df.loc[student_df["grade"]=="12th"].groupby(["school_name"])
twelveth_math_score=twelveth["math_score"].mean()


In [326]:
# Creating summary of Math scores by Grade for students of each grade level (9th, 10th, 11th, 12th) at each school.
math_score_grade= pd.DataFrame({"9th":nineth_math_score, "10th":tenth_math_score,
                   "11th":eleventh_math_score, "12th":twelveth_math_score
                    }, columns=["9th","10th","11th","12th"])
math_score_grade.head(15)

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [329]:
#7.------ Reading Scores by Grade
# First using loc to filter per grade, then groupby per school
# Second getting column of reading score

# 9th grade
nineth= student_df.loc[student_df["grade"]=="9th"].groupby(["school_name"])
nineth_read_score=nineth["reading_score"].mean()
# 10th grade
tenth= student_df.loc[student_df["grade"]=="10th"].groupby(["school_name"])
tenth_read_score=tenth["reading_score"].mean()
# 11th grade
eleventh= student_df.loc[student_df["grade"]=="11th"].groupby(["school_name"])
eleventh_read_score=eleventh["reading_score"].mean()
# 12th grade
twelveth= student_df.loc[student_df["grade"]=="12th"].groupby(["school_name"])
twelveth_read_score=twelveth["reading_score"].mean()

In [330]:
# Creating a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
read_score_grade= pd.DataFrame({"9th":nineth_read_score, "10th":tenth_read_score,
                   "11th":eleventh_read_score, "12th":twelveth_read_score
                    }, columns=["9th","10th","11th","12th"])
read_score_grade.head(15)

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [278]:
# 8. -----Scores by School Spending

#Binning 
spen_ran_bins= [0,585,630,645,680 ]
spen_ran_labels= ["<$585","$585-630","$630-645","$645-680"]
spend_ranges_df=schools_summary

In [331]:
# Creting binns for rthe DataFrame
spend_ranges_df["Spending Ranges(Per Student)"]= pd.cut(student_budget,spen_ran_bins,labels=spen_ran_labels)

In [333]:
#Spending Ranges(Per Student) Groupping by our bins

spent_math= spend_ranges_df.groupby("Spending Ranges(Per Student)").mean()["Average Math Score"]

per_math= spend_ranges_df.groupby("Spending Ranges(Per Student)").mean()["% Passing Math"]

per_reading= spend_ranges_df.groupby("Spending Ranges(Per Student)").mean()["% Passing Reading"]

spent_reading= spend_ranges_df.groupby("Spending Ranges(Per Student)").mean()["Average Reading Score"]

spent_overall= spend_ranges_df.groupby("Spending Ranges(Per Student)").mean()["Overall Passing Rate"]


# Create a table that breaks down school performances based on average Spending Ranges (Per Student). Four reasonable bins were created to group school spending.
school_spending_summary=pd.DataFrame({"Average Math Score":spent_math,
                                      "Average Reading Score":spent_reading,
                                      "% Passing Math":per_math,
                                      "% Passing Reading":per_reading,
                                      "Overall Passing Rate":spent_overall},
                                    columns=["Average Math Score","Average Reading Score",
                                    "% Passing Math","% Passing Reading","Overall Passing Rate"])

# Improve formatting before outputting
school_spending_summary["Average Math Score"]=school_spending_summary["Average Math Score"].map("{0:,.2f}".format)
school_spending_summary["Average Reading Score"]=school_spending_summary["Average Reading Score"].map("{0:,.2f}".format)
school_spending_summary["% Passing Math"]=school_spending_summary["% Passing Math"].map("{0:,.2f}".format)
school_spending_summary["% Passing Reading"]=school_spending_summary["% Passing Reading"].map("{0:,.2f}".format)
school_spending_summary["Overall Passing Rate"]=school_spending_summary["Overall Passing Rate"].map("{0:,.2f}".format)



In [289]:
school_spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges(Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


In [299]:
# 9. -----------Scores by School Size
#Binning 
bins= [0,1000,2000,5000]
labels= ["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]

# DataFrame
size_ranges= schools_summary

In [300]:
# Creting binns for rthe DataFrame
size_ranges["School Size"]= pd.cut(total_students,bins,labels=labels)

In [301]:
# Scores by School Size, Groupping by our bins

ave_math= size_ranges.groupby("School Size").mean()["Average Math Score"]

ave_reading= size_ranges.groupby("School Size").mean()["Average Reading Score"]

size_per_math= size_ranges.groupby("School Size").mean()["% Passing Math"]

size_per_reading= size_ranges.groupby("School Size").mean()["% Passing Reading"]

size_per_overall= size_ranges.groupby("School Size").mean()["Overall Passing Rate"]

score_school_size_summary=pd.DataFrame({"Average Math Score":ave_math,
                                      "Average Reading Score":ave_reading,
                                      "% Passing Math":size_per_math,
                                      "% Passing Reading":size_per_reading,
                                      "Overall Passing Rate":size_per_overall},
                                    columns=["Average Math Score","Average Reading Score",
                                    "% Passing Math","% Passing Reading","Overall Passing Rate"])


In [302]:
score_school_size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


In [334]:
# 10. ------Scores by School Type

# DataFrame with information to use
type_school= schools_summary

In [335]:
# Creting the parameters for the Summary of Scores by School Type

ave_math_typ= type_school.groupby("School Type").mean()["Average Math Score"]

ave_reading_type= type_school.groupby("School Type").mean()["Average Reading Score"]

per_math= type_school.groupby("School Type").mean()["% Passing Math"]

per_reading= type_school.groupby("School Type").mean()["% Passing Reading"]

per_overall= type_school.groupby("School Type").mean()["Overall Passing Rate"]

score_school_type_summary=pd.DataFrame({"Average Math Score":ave_math_typ,
                                      "Average Reading Score":ave_reading_type,
                                      "% Passing Math":per_math,
                                      "% Passing Reading":per_reading,
                                      "Overall Passing Rate":per_overall},
                                    columns=["Average Math Score","Average Reading Score",
                                    "% Passing Math","% Passing Reading","Overall Passing Rate"])


In [336]:
# Printing the summary
score_school_type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
